<a href="https://colab.research.google.com/github/HarishSidagam8/Deep-Learning-Projects/blob/main/DL_Project_8_Breast_Cancer_Prediction_Using_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cpu')

In [5]:
# Data collection and preprocessing
data = load_breast_cancer()
x, y = data.data, data.target

In [6]:
print(x,y)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

In [11]:
# splitting featrures and target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
print(x.shape, x_train.shape, x_test.shape)

(569, 30) (455, 30) (114, 30)


In [14]:
# Standardizing data
scalar=StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)

In [15]:
type(x_train)

numpy.ndarray

In [28]:
#convert the data to pytorch tensors and move to gpu
x_train=torch.tensor(x_train).to(device)
x_test=torch.tensor(x_test).to(device)
y_train=torch.tensor(y_train, dtype=torch.float32).to(device)
y_test=torch.tensor(y_test, dtype=torch.float32).to(device)

/tmp/ipython-input-3263378762.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train=torch.tensor(x_train).to(device)
/tmp/ipython-input-3263378762.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test=torch.tensor(x_test).to(device)
/tmp/ipython-input-3263378762.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train=torch.tensor(y_train, dtype=torch.float32).to(device)
/tmp/ipython-input-3263378762.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

In [29]:
# Neural Network Archietecture
class BreastCancerClassifier(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(BreastCancerClassifier, self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,output_size)
        self.sigmoid=nn.Sigmoid()
    def forward(self,x):
        out=self.fc1(x)
        out=self.relu(out)
        out=self.fc2(out)
        out=self.sigmoid(out)
        return out


In [30]:
# Define Hyperparameters
input_size=x_train.shape[1]
hidden_size=64
output_size=1
learning_rate=0.001
num_epochs=100
batch_size=32

In [31]:
# Intilaize the model
model=BreastCancerClassifier(input_size,hidden_size,output_size).to(device)

In [32]:
#define the loss and optimizer
criterion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [33]:
# Training the neural network
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs=model(x_train)
  loss=criterion(outputs,y_train.view(-1,1))
  loss.backward()
  optimizer.step()

In [37]:
# claculate accuracy
with torch.no_grad():
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1, 1)).sum().item()
  accuracy = correct / y_train.size(0)

  if (epoch + 1) % 10 == 0:
    print(f'Epoch |{epoch+1}/{num_epochs}|, Loss :{loss.item():.4f},Accuracy: {accuracy*100:.2f}%')

Epoch |100/100|, Loss :0.1129,Accuracy: 97.36%


In [39]:
# modelEvaluation
model.eval()
with torch.no_grad():
  outputs=model(x_train)
  predicted=outputs.round()
  correct=(predicted==y_train.view(-1,1)).float().sum()
  accuracy=correct/y_train.size(0)
  print(f'Train Accuracy: {accuracy*100:.2f}%')

Train Accuracy: 97.36%


In [40]:
# Evaluation on test data
model.eval()
with torch.no_grad():
  outputs=model(x_test)
  predicted=outputs.round()
  correct=(predicted==y_test.view(-1,1)).float().sum()
  accuracy=correct/y_test.size(0)
  print(f'Test Accuracy: {accuracy*100:.2f}%')

Test Accuracy: 97.37%
